#### Coutresy to convert XML to JSON https://stackoverflow.com/questions/191536/converting-xml-to-json-using-python 
Made small modifications for our purpose
Remove certain first few lines from XML files and run the code by changing the XML filename

In [1]:
from xml.dom import minidom
import simplejson as json
def parse_element(element):
    dict_data = dict()
    if element.nodeType == element.TEXT_NODE:
        if element.data != " ":
            dict_data['data'] = element.data
    if element.nodeType not in [element.TEXT_NODE, element.DOCUMENT_NODE, 
                                element.DOCUMENT_TYPE_NODE]:
        for item in element.attributes.items():
            dict_data[item[0]] = item[1]
    if element.nodeType not in [element.TEXT_NODE, element.DOCUMENT_TYPE_NODE]:
        for child in element.childNodes:
            child_name, child_dict = parse_element(child)
            if child_name in dict_data:
                try:
                    dict_data[child_name].append(child_dict)
                except AttributeError:
                    dict_data[child_name] = [dict_data[child_name], child_dict]
            else:
                dict_data[child_name] = child_dict 
    return element.nodeName, dict_data

if __name__ == '__main__':
    dom = minidom.parse('sample1.xml')
    print type(dom)
    f = open('data12.json', 'w')
    f.write(json.dumps(parse_element(dom), sort_keys=True, indent=4))
    f.close()

<type 'instance'>


In [2]:
import pandas as pd

data = pd.read_json('data12.json')
df3 = pd.io.json.json_normalize(data[0][1]['xml']['Thread'])
df4 = pd.concat([pd.DataFrame(pd.io.json.json_normalize(x)) for x in df3['RelComment']],ignore_index=True)
df4['THREAD_SEQUENCE'] = [ '_'.join(i.split('_')[:2]) for i in df4['RELC_ID']]
df5 = pd.concat([df3.set_index('THREAD_SEQUENCE'),df4.set_index('THREAD_SEQUENCE')], axis=1, join='inner').reset_index()

[ df5.drop(i, axis=1, inplace=True) for i in ['RelQuestion.#text' , 'RelComment', 'RelCBody.#text.data' ,
                                              'RelQuestion.RelQBody.#text.data' , 'RelQuestion.RelQBody.#text.data', '#text'] ]
#df5[ df5['THREAD_SEQUENCE'] == 'Q268_R16' ]
df5.describe()

,THREAD_SEQUENCE,RelQuestion.RELQ_CATEGORY,RelQuestion.RELQ_DATE,RelQuestion.RELQ_ID,RelQuestion.RELQ_USERID,RelQuestion.RELQ_USERNAME,RelQuestion.RelQClean.#text.data,RelQuestion.RelQSubject.#text.data,RELC_DATE,RELC_ID,RELC_RELEVANCE2RELQ,RELC_USERID,RELC_USERNAME,RelCClean.#text.data
count,2440,2440,2440,2440,2440,2440,2440,2440,2440,2440,2440,2440,2440,2440
unique,244,21,244,244,196,196,244,243,2434,2440,3,914,914,2401
top,Q295_R63,Qatar Living Lounge,2013-02-24 09:50:09,Q295_R63,U2,anonymous,Do you know any backbiters in your office? // ...,QP Offer,2011-07-28 14:25:18,Q269_R7_C7,Bad,U2,anonymous,very good but i thing expensive
freq,10,860,10,10,310,310,10,20,3,1,1209,374,374,10
